In [13]:

!wget -O flowers.zip "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
!mkdir -p /content/flowers_raw
!tar -xzf flowers.zip -C /content/flowers_raw


--2025-12-21 13:22:06--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.207, 108.177.98.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flowers.zip’

flowers.zip         100%[===================>] 218.21M   198MB/s    in 1.1s    

2025-12-21 13:22:07 (198 MB/s) - ‘flowers.zip’ saved [228813984/228813984]



In [8]:
import os, shutil, random
from pathlib import Path

raw_dir = Path("/content/flowers_raw/flower_photos")
base_dir = Path("/content/flowers_dataset")
train_dir = base_dir / "train"
test_dir  = base_dir / "test"

classes = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]


for split in ["train", "test"]:
    for cls in classes:
        (base_dir / split / cls).mkdir(parents=True, exist_ok=True)

def split_class(cls_name, train_ratio=0.8):
    src = raw_dir / cls_name
    files = [f for f in os.listdir(src) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    random.shuffle(files)

    split_idx = int(len(files) * train_ratio)
    train_files = files[:split_idx]
    test_files  = files[split_idx:]

    for f in train_files:
        shutil.copy(src / f, train_dir / cls_name / f)
    for f in test_files:
        shutil.copy(src / f, test_dir / cls_name / f)

for c in classes:
    split_class(c)

for c in classes:
    print(c, "=> train:", len(os.listdir(train_dir / c)), " test:", len(os.listdir(test_dir / c)))


daisy => train: 506  test: 127
dandelion => train: 718  test: 180
roses => train: 512  test: 129
sunflowers => train: 559  test: 140
tulips => train: 639  test: 160


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (160, 160)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)


Found 2934 images belonging to 5 classes.
Found 736 images belonging to 5 classes.


In [10]:
num_classes = len(classes)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    epochs=15,
    validation_data=test_gen
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.3175 - loss: 1.6344 - val_accuracy: 0.5448 - val_loss: 1.1482
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 221s 2s/step - accuracy: 0.5675 - loss: 1.0944 - val_accuracy: 0.6196 - val_loss: 1.0333
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 237s 3s/step - accuracy: 0.6075 - loss: 1.0030 - val_accuracy: 0.6549 - val_loss: 0.9497
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 221s 2s/step - accuracy: 0.6603 - loss: 0.9102 - val_accuracy: 0.6495 - val_loss: 0.8919
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.6725 - loss: 0.8481 - val_accuracy: 0.6712 - val_loss: 0.8140
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.7068 - loss: 0.7764 - val_accuracy: 0.6848 - val_loss: 0.7868
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.7170 - loss: 0.7653 - val_accuracy: 0.7106 - val_loss: 0.7643
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.7297 - loss: 0.7208 - val_accuracy: 0.7038 - v

In [14]:
import numpy as np
from tensorflow.keras.preprocessing import image
from pathlib import Path
import os


daisy_test_path = test_dir / "daisy"
daisy_images = [f for f in os.listdir(daisy_test_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if daisy_images:
    sample_path = daisy_test_path / daisy_images[0]
    print(f"Using sample image: {sample_path}")
else:
    print(f"Error: No image files found in {daisy_test_path}. Cannot perform prediction.")

    raise FileNotFoundError(f"No image files found in {daisy_test_path}")

img = image.load_img(sample_path, target_size=IMG_SIZE)
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis=0) / 255.0

pred = model.predict(img_arr)[0]
pred_idx = np.argmax(pred)
pred_class = classes[pred_idx]

print("Image:", sample_path)
print("Predicted class:", pred_class)
print("Class indices:", train_gen.class_indices)

Using sample image: /content/flowers_dataset/test/daisy/5561775629_a2b709b3a4_n.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Image: /content/flowers_dataset/test/daisy/5561775629_a2b709b3a4_n.jpg
Predicted class: daisy
Class indices: {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
